In [2]:
import scipy
import scipy.sparse
import cPickle
import logging
import implicit
reload(implicit)
import implicit.approximate_als
reload(implicit.approximate_als)
import implicit.bpr
reload(implicit.bpr)
import sklearn
import numpy as np
import itertools
import random
import pandas as pd
import time
import lightfm
import lightfm.evaluation
import tqdm
import util as C
reload(C)
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

# Load Data

In [3]:
logging.basicConfig(level = logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [4]:
user_helper = C.get_user_helper()

In [5]:
df_idx = user_helper.load_subs_idx_df()

In [6]:
df_idx.head(3)

,cn,lang,br,cs
0,jp,ja,docomo,48285
1,cn,zh,google,2781+115474
2,us,en,xiaomi,97197


In [7]:
%%time
mat_coo = C.load_mat(C.USER_ITEM_MAT_PATH)

CPU times: user 388 ms, sys: 208 ms, total: 596 ms
Wall time: 596 ms


In [8]:
%%time
mat_csr = mat_coo.tocsr()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16 µs


# Split Train/Test Data

In [9]:
MIN_SUBS = 6
MAX_SUBS = 10
TRAIN_USERS = 20 * 10000
train_mat_csr, train_users_idx = C.build_train_matrix(mat_csr, n_users = TRAIN_USERS, min_subs=MIN_SUBS, max_subs=MAX_SUBS)

  1%|          | 1313/200000 [00:00<00:15, 13121.47it/s]

# of all users = 4609357
# of ok users = 1054508
# of train users = 200000


100%|██████████| 200000/200000 [00:14<00:00, 13348.85it/s]


In [10]:
%%time
train_mat_coo = train_mat_csr.tocoo()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.4 ms


In [11]:
train_meta_df = df_idx.iloc[train_users_idx].copy()
train_meta_df['subs'] = train_meta_df['cs'].astype(str).apply(lambda x: len(x.split('+')))
train_meta_df.reset_index(inplace=True)
train_meta_df.drop(['cs'], axis=1, inplace=True)
train_meta_df.rename_axis({'index':'origin_idx'}, axis = 1, inplace=True)

In [12]:
train_meta_df.head(3)

,origin_idx,cn,lang,br,subs
0,3348456,us,en,samsung,8
1,2461395,ru,ru,samsung,6
2,2281385,pt,pt,asus,6


In [13]:
train_meta_df.to_csv(C.TRAIN_USER_ITEM_META_PATH)

In [14]:
!head data/train_user_item_meta.csv

,origin_idx,cn,lang,br,subs
0,3348456,us,en,samsung,8
1,2461395,ru,ru,samsung,6
2,2281385,pt,pt,asus,6
3,4378970,us,en,samsung,6
4,2562956,us,en,lge,10
5,3333271,es,es,samsung,6
6,1775689,au,en,huawei,9
7,3887295,us,en,samsung,9
8,3569011,us,en,essential,9


In [23]:
TEST_USERS = 5 * 10000
train_mat_csr2, test_mat_csr, test_users_idx = C.train_test_split(train_mat_csr, n_test_users=TEST_USERS)

  2%|▏         | 1069/50000 [00:00<00:04, 10682.40it/s]

# of test users = 50000


100%|██████████| 50000/50000 [00:04<00:00, 10890.42it/s]


In [24]:
uid_idx = test_users_idx[1]
train_mat_csr2[uid_idx].indices, test_mat_csr[uid_idx].indices, train_mat_csr[uid_idx].indices

(array([11954, 15500, 25981], dtype=int32),
 array([19038, 26322, 68851], dtype=int32),
 array([11954, 15500, 19038, 25981, 26322, 68851], dtype=int32))

In [25]:
test_meta_df= train_meta_df.iloc[test_users_idx]
test_meta_df.to_csv(C.TEST_USER_ITEM_META_PATH)

In [26]:
test_meta_df.head(3)

,origin_idx,cn,lang,br,subs
119737,2102869,cu,en,huawei,6
72272,2026103,at,fr,samsung,6
158154,1385691,us,en,samsung,6


In [28]:
test_meta_df.shape

(50000, 5)

In [27]:
!head data/test_user_item_meta.csv

,origin_idx,cn,lang,br,subs
119737,2102869,cu,en,huawei,6
72272,2026103,at,fr,samsung,6
158154,1385691,us,en,samsung,6
65426,4125898,us,en,google,6
30074,860652,us,en,samsung,6
23677,2845547,co,es,samsung,9
134858,1013774,us,en,verizon,8
176418,3620526,pt,pt,huawei,10
132467,990024,us,en,samsung,7


# Save Data

In [29]:
C.save_mat(C.TRAIN_USER_ITEM_MAT_PATH, train_mat_csr2, compressed=False)
C.save_mat(C.TEST_USER_ITEM_MAT_PATH, test_mat_csr, compressed=False)